In [1]:
import os

import numpy as np
import xlsxwriter
import psycopg2
import pandas as pd
import pandas.io.sql as psql
import re

from aggregation import Aggregation
from solver import Solver

In [20]:
# estimates = [[10, 12, 4],
#              [15, 12, 3],
#              [15, 8, 2],
#              [5, 16, 1],
#              [20, 4, 2]],
# direction = ['min', 'min', 'max']

estimates = [[3, 4, 2],
             [2, 3, 4],
             [3, 3, 2],
             [2, 4, 4]],
direction = ['max', 'max', 'max']
coefficients = [1, 1, 1]
#
# estimates = [[210, 27, 350, 30, 30, 1080, 30, 4, 10979],
#              [570, 34, 6000, 5000, 19, 2160, 60, 4.8, 159599],
#              [249, 31, 6000, 4000, 16, 2160, 30, 4.9, 86488],
#              [245, 22, 500, 120, 10, 2160, 15, 4.6, 21990],
#              [240, 15, 100, 100, 12, 1080, 30, 3.9, 3700],
#              [249, 22, 500, 120, 9, 1080, 30, 5, 14990],
#              [249, 30, 4000, 3000, 13, 1080, 60, 4.5, 49170],
#              [249, 20, 300, 120, 5, 1080, 30, 3.9, 5190],
#              [595, 31, 6000, 5000, 19, 2160, 60, 4.7, 132890],
#              [80, 13, 20, 10, 8, 720, 30, 4.6, 22625]]
# direction = ['min', 'max', 'max', 'max', 'max', 'max', 'max', 'max', 'min']
# coefficients = [2, 2, 1, 1, 1, 2, 2, 1, 3]

In [29]:
preferences_matrices = Aggregation.get_preferences_matrices(estimates, direction)
Q_arithmetic = Aggregation.Q_arithmetic(preferences_matrices, coefficients)
d_arith = Aggregation.calc_distance(preferences_matrices, Q_arithmetic)
Q_median = Aggregation.Q_median(preferences_matrices, coefficients)
d_median = Aggregation.calc_distance(preferences_matrices, Q_median)
Q_minimax = Aggregation.Q_minimax(preferences_matrices, coefficients)
d_minimax = Aggregation.calc_distance(preferences_matrices, Q_minimax)

l = [(0, 'Среднее арифметическое', d_arith, Q_arithmetic), (1, 'Медиана', d_median, Q_median),
     (2, 'Минимакс', d_minimax, Q_minimax)]
with_min_d = min(l, key=lambda x: x[0])

print()

print(d_arith, d_median, d_minimax)
# arith = Aggregation.get_levels_with(Q_arithmetic)
# median = Aggregation.get_levels_with(Q_median)
# minimax = Aggregation.get_levels_with(Q_minimax)

0.3175812547241118
0.3634467120181405
0.32904761904761903
(0.3175812547241118, array([[0.5       , 0.4984127 , 0.47619048, 0.52222222],
       [0.5015873 , 0.5       , 0.47777778, 0.52380952],
       [0.52380952, 0.52222222, 0.5       , 0.54603175],
       [0.47777778, 0.47619048, 0.45396825, 0.5       ]]))
0.3175812547241118 0.3634467120181405 0.32904761904761903


In [441]:
print('P_sum\n')
total = Aggregation.get_levels_with(total_preferences_matrix)
print('_________________________________________________________________________\n')
print('Q_arithmetic\n')
arithmetic = Aggregation.get_levels_with(Q_arithmetic)
print('_________________________________________________________________________\n')
print('Q_median\n')
median = Aggregation.get_levels_with(Q_median)
print('_________________________________________________________________________\n')
print('Q_minimax\n')
minimax = Aggregation.get_levels_with(Q_minimax)
print('_________________________________________________________________________\n')

P_sum

Граф содержит контуры, нельзя разбить на уровни с помощью адгоритма Демукрона

Процедура Коупленда:
[ 3  1  5 -7  1 -5  9 -1  3 -9]
[[7], [3], [1, 9], [2, 5], [8], [6], [4], [10]]

Процедура Разности Весов:
[  4.28041115   2.0251474    6.2835196   -6.02828225   2.72633617
  -2.38914097   8.52389928   1.82579663   2.1793191  -19.4270061 ]
[[7], [3], [1], [5], [9], [2], [8], [6], [4], [10]]

Процедура Отношения Весов:
[ 5.7198433   2.01410103 20.54759538  0.22551064  1.99631778  0.5488305
         inf  1.53051942  2.24808526  0.        ]
[[7], [3], [1], [9], [2], [5], [8], [6], [4], [10]]
_________________________________________________________________________

Q_arithmetic

Граф содержит контуры, нельзя разбить на уровни с помощью адгоритма Демукрона

Процедура Коупленда:
[ 3  1  5 -7  1 -5  9 -1  3 -9]
[[7], [3], [1, 9], [2, 5], [8], [6], [4], [10]]

Процедура Разности Весов:
[ 0.47560124  0.22501638  0.69816884 -0.66980914  0.30292624 -0.26546011
  0.94709992  0.20286629  0.24

/var/folders/23/hzkz91rj3pq502cfq0_65r0c0000gn/T/ipykernel_25373/3516334113.py:141: RuntimeWarning: divide by zero encountered in true_divide
  c = np.sum(matrix, 0) / np.sum(matrix, 1)


# Чтение из базы

In [104]:
DB_CFG = {
    'dbname': 'diploma',
    'user': 'diploma',
    'password': 'diploma',
    'host': 'localhost'
}

In [217]:
conn = psycopg2.connect(**DB_CFG)
df_helicopters = psql.read_sql('''SELECT name,
                               weight,
                               duration,
                               distance,
                               height,
                               speed,
                               pixels,
                               fps,
                               rating,
                               price
                        FROM helicopters
                        ORDER BY id''', conn)

In [218]:
conn = psycopg2.connect(**DB_CFG)
df_criteria = psql.read_sql('''SELECT num, name, coefficient, direction
                        FROM criteria
                        ORDER BY num''', conn)

In [268]:
solver = Solver(df_helicopters, df_criteria)

In [269]:
solver.get_xls([[0, 1], [0, 1, 2], [0, 1, 5, 6, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8]], 'test.xlsx')

[[0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 1 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0]
 [1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1 0]
 [0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0]
 [1 1 0 1 1 1 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 1 0]
 [1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 0]
 [0 1 0 1 1 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1 0]
 [1 1 0 1 1 1 0 0 1 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 1 0]
 [1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0]
 [0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 1 0]
 [1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 0 1 1

/var/folders/23/hzkz91rj3pq502cfq0_65r0c0000gn/T/ipykernel_98525/679168550.py:141: RuntimeWarning: divide by zero encountered in true_divide
  c = np.sum(matrix, 0) / np.sum(matrix, 1)


In [293]:
filename = 'input20220509202749.xlsx'

In [296]:
df_helicopters_from_file = pd.read_excel(f'ruby/public/uploads/{filename}', sheet_name=0)
criteria_temp = pd.read_excel(f'ruby/public/uploads/{filename}', sheet_name=1)
df_criteria_from_file = pd.DataFrame(criteria_temp.to_numpy().T, columns=['name', 'coefficient', 'direction'])
df_criteria_from_file.insert(0, 'num', [x + 1 for x in range(len(df_criteria_from_file))])
df_groups_from_file = pd.read_excel(f'ruby/public/uploads/{filename}', sheet_name=2)
groups_from_file = list(
    map(lambda x: list(dict.fromkeys(list(map(lambda x: int(x) - 1, re.sub("[^0-9,]", "", x[0]).split(','))))),
        list(df_groups_from_file.to_numpy())))

In [298]:
solver = Solver(df_helicopters_from_file, df_criteria_from_file)
solver.get_xls(groups_from_file, 'output.xlsx')

[[0 1 0 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [1 1 0 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 0 1 0 0 1 0 1 1 0 0 0 0]
 [1 1 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 0 1]
 [1 1 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0]
 [1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1]
 [1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 0 1]
 [1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0

/var/folders/23/hzkz91rj3pq502cfq0_65r0c0000gn/T/ipykernel_98525/679168550.py:141: RuntimeWarning: divide by zero encountered in true_divide
  c = np.sum(matrix, 0) / np.sum(matrix, 1)


In [272]:
df_groups_from_file = pd.read_excel(f'input.xlsx', sheet_name=2)

In [285]:
df_groups_from_file = list(map(lambda x: list(map(lambda x: int(x), re.sub("[^0-9,]", "", x[0]).split(','))),
                               list(df_groups_from_file.to_numpy())))

[[1, 2], [1, 2, 3], [1, 2, 6, 7, 9]]

In [18]:
a = [
    np.array([[1, 2], [3, 4]]),
    np.array([[2, 2], [3, 4]]),
    np.array([[1, 2], [3, 5]]),
    np.array([[1, 2], [2, 5]]),
]
b = np.array([[1, 2], [3, 3]])

In [14]:
def calc_distance(preferences_matrices, Q_matrix):
    distance = 0
    for m in preferences_matrices:
        distance += np.square(Q_matrix - m).sum()

    return distance

In [2]:
os.path.dirname(__file__)

NameError: name '__file__' is not defined

In [4]:
alternatives_names = ['Andrey', 'Max', 'Stepan']
estimates = np.array([[5, 8], [12, 4], [5, 3]])

In [5]:
df_alternatives = pd.concat([pd.DataFrame({'name': alternatives_names}), pd.DataFrame(estimates)], axis=1)